<a href="https://colab.research.google.com/github/draaronv/statistical_learning/blob/master/classication_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('googledrive')
install.packages('httpuv') 
library('googledrive') 
library('httpuv')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘promises’, ‘later’




In [ ]:
if (file.exists("/usr/local/lib/python3.7/dist-packages/google/colab/_ipython.py")) {
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check) 
  options(rlang_interactive=TRUE)
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.1 (2020-08-26 16:20:06 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.24.0 (2020-08-26 16:11:58 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.11.0 (2021-09-26 08:30:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, inherits, isOpen, nullfile, parse,
    warnings




In [ ]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/auth?client_id=603366585132-dpeg5tt0et3go5of2374d83ifevk5086.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code



Enter authorization code: 4/1AX4XfWguo5LMKPA5fUIUkUJW05t4yD4Ky5dBnTkHTXpuWeuXWXq5rQoY79w


In [ ]:
x <- drive_get("~/statistical_learning/finalData.csv")
drive_download(x)





✔ The input `path` resolved to exactly 1 file.

File downloaded:

• finalData.csv <id: 1W1wazYCqs5Iiz6cAwFUjo9f3a3kaBxWU>

Saved locally as:

• finalData.csv



In [ ]:
rm(list = ls())

In [ ]:
install.packages("Metrics")
install.packages("rsample")
install.packages("rpart")
install.packages("dplyr")
install.packages("rpart.plot")
install.packages("ipred")
install.packages("caret")
install.packages("caTools")
install.packages("ranger")
library(ranger)
library(Metrics)
library(rsample)     # data splitting 
library(dplyr)       # data wrangling
library(rpart)       # performing regression trees
library(rpart.plot)  # plotting regression trees
library(ipred)       # bagging
library(caret)       # bagging
library(caTools)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘Metrics’


The following object is masked from ‘package:R.oo’:

    ll



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:



In [ ]:
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

# Data Initilization

In [ ]:
data1 = read.csv("finalData.csv")

In [ ]:
data2=head(data1,50000)

In [ ]:
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 50000

[1] 49917

[1] 49917

'data.frame':	39219 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 8 levels "Clear","Clouds",..: 1 1 1 1 1 1

## Ranger

In [ ]:
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)

In [ ]:
print(randFore100m5)

Ranger result

Call:
 ranger(KILLED_INJURED ~ ., data = training, mtry = 13, num.trees = 100,      min.node.size = 2) 

Type:                             Classification 
Number of trees:                  100 
Sample size:                      39219 
Number of independent variables:  13 
Mtry:                             13 
Target node size:                 2 
Variable importance mode:         none 
Splitrule:                        gini 
OOB prediction error:             20.80 % 


In [ ]:
pred=predict(randFore100m5,test,type="response")

In [ ]:
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

Confusion Matrix and Statistics

    predic
true    0    1    2    3    4    5    6    7    8   10   12   18
  0  7331  554  163   64   16    5   10    0    0    3    0    4
  1   930  743   25   12    9    2    1    1    0    0    0    0
  2   271   32  182   12    2    0    0    0    0    0    0    0
  3    91   10    4   59    1    0    0    0    0    0    0    0
  4    37    9    3    0   34    1    0    0    0    0    0    0
  5    19    1    1    0    0   21    1    0    0    0    0    0
  6     7    3    0    0    0    0    3    0    0    0    0    0
  7     3    2    0    0    0    0    0    1    0    0    0    0
  8     0    0    0    0    0    0    0    0    5    0    0    0
  10    1    0    0    0    0    0    0    0    0    2    0    0
  12    0    0    0    0    0    0    0    0    0    0    3    0
  18    1    1    0    0    0    0    0    0    0    0    0    2

Overall Statistics
                                         
               Accuracy : 0.7839         
       

### Grid Optimization

In [ ]:
hyper_grid <- expand.grid(
  mtry       = 13,
  node_size  = 2,
  num.trees = seq(100,1000, by = 100),
  OOB_RMSE   = 0
)
for(i in 1:nrow(hyper_grid)) {
  model <- ranger(
    formula         = KILLED_INJURED~., 
    data            = training, 
    num.trees       = hyper_grid$num.trees[i],
    mtry            = hyper_grid$mtry[i],
    min.node.size   = hyper_grid$node_size[i],
    seed            = 123
  )
  
  # add OOB error to grid
  hyper_grid$OOB_RMSE[i] <- sqrt(model$prediction.error)
}


Growing trees.. Progress: 80%. Estimated remaining time: 7 seconds.
Growing trees.. Progress: 52%. Estimated remaining time: 28 seconds.
Growing trees.. Progress: 39%. Estimated remaining time: 47 seconds.
Growing trees.. Progress: 80%. Estimated remaining time: 15 seconds.
Growing trees.. Progress: 32%. Estimated remaining time: 1 minute, 5 seconds.
Growing trees.. Progress: 65%. Estimated remaining time: 32 seconds.
Growing trees.. Progress: 98%. Estimated remaining time: 1 seconds.
Growing trees.. Progress: 27%. Estimated remaining time: 1 minute, 24 seconds.
Growing trees.. Progress: 54%. Estimated remaining time: 52 seconds.
Growing trees.. Progress: 82%. Estimated remaining time: 20 seconds.
Growing trees.. Progress: 22%. Estimated remaining time: 1 minute, 47 seconds.
Growing trees.. Progress: 45%. Estimated remaining time: 1 minute, 14 seconds.
Growing trees.. Progress: 68%. Estimated remaining time: 43 seconds.
Growing trees.. Progress: 91%. Estimated remaining time: 12 second

In [ ]:
hyper_grid %>% 
  dplyr::arrange(OOB_RMSE) %>%
  head(10)

,mtry,node_size,num.trees,OOB_RMSE
,<dbl>,<dbl>,<dbl>,<dbl>
1,13,2,500,0.4483202
2,13,2,600,0.4486184
3,13,2,800,0.4487888
4,13,2,700,0.4489023
5,13,2,1000,0.4492001
6,13,2,400,0.4492143
7,13,2,900,0.4493844
8,13,2,300,0.4499651
9,13,2,200,0.4505309


In [ ]:
randFore <- ranger(KILLED_INJURED~., data=training, mtry=14,num.trees=700,min.node.size =2)
pred=predict(randFore,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8    10    12    18
  0  14586   567   170    63    26     7     6     1     1     2     1     0
  1   1000  2017    38     9     4     2     0     1     1     1     0     0
  2    291    47   552     4     0     0     1     0     0     0     0     0
  3     90    16     2   178     3     0     0     0     0     0     0     0
  4     29     8     0     1   114     0     0     0     0     0     0     0
  5      8     1     1     0     2    60     2     0     0     0     0     0
  6      6     2     0     0     0     0    27     0     0     0     0     0
  7      0     2     1     0     0     0     0     5     0     0     0     0
  8      1     0     0     0     0     0     0     0     7     0     0     0
  10     0     0     0     0     0     0     0     0     0     4     0     0
  12     0     0     0     0     0     0     0     0     0     0    10     0
  18     0     0     0     0    

In [ ]:
data2=head(data1,100000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
hyper_grid <- expand.grid(
  mtry       = 13,
  node_size  = 2,
  num.trees = seq(100,700, by = 100),
  OOB_RMSE   = 0
)
for(i in 1:nrow(hyper_grid)) {
  model <- ranger(
    formula         = KILLED_INJURED~., 
    data            = training, 
    num.trees       = hyper_grid$num.trees[i],
    mtry            = hyper_grid$mtry[i],
    min.node.size   = hyper_grid$node_size[i],
    seed            = 123
  )
  
  # add OOB error to grid
  hyper_grid$OOB_RMSE[i] <- sqrt(model$prediction.error)
}
hyper_grid %>% 
  dplyr::arrange(OOB_RMSE) %>%
  head(10)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 100000

[1] 99917

[1] 99917

'data.frame':	78506 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 10 levels "Clear","Clouds",..: 1 1 1 1 1 

,mtry,node_size,num.trees,OOB_RMSE
,<dbl>,<dbl>,<dbl>,<dbl>
1,13,2,500,0.4483202
2,13,2,600,0.4486184
3,13,2,700,0.4489023
4,13,2,400,0.4492143
5,13,2,300,0.4499651
6,13,2,200,0.4505309
7,13,2,100,0.4526464


In [ ]:
randFore <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=500,min.node.size =2)
pred=predict(randFore,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

Growing trees.. Progress: 32%. Estimated remaining time: 1 minute, 4 seconds.
Growing trees.. Progress: 65%. Estimated remaining time: 32 seconds.
Growing trees.. Progress: 98%. Estimated remaining time: 1 seconds.


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8    10    12    18
  0  15011  1101   273   131    44    18    15     1     1     3     0     4
  1   1820  1342    56    22    13     3     2     1     1     2     0     0
  2    508    60   284    14     5     0     2     0     0     0     0     0
  3    198    22     7   109     1     1     0     0     0     0     0     0
  4    106    19     3     0    64     1     0     0     0     0     0     0
  5     41     1     1     0     0    27     0     0     0     0     0     0
  6     17     8     0     0     1     0    10     0     0     0     0     0
  7      4     2     0     0     0     0     0     2     0     0     0     0
  8      4     0     0     0     0     0     0     0     5     0     0     0
  10     3     0     0     0     0     0     0     0     0     2     0     0
  12     2     0     0     0     0     0     0     0     0     0     9     0
  18     1     1     0     0    

# Different Sizes

In [ ]:
## Data pre-processing
data2=head(data1,100000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 100000

[1] 99917

[1] 99917

'data.frame':	78506 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 10 levels "Clear","Clouds",..: 1 1 1 1 1 

Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8    10    12    18
  0  14959  1138   287   132    44    17    16     1     1     3     0     4
  1   1811  1353    52    23    16     2     1     1     1     2     0     0
  2    499    65   288    14     5     0     2     0     0     0     0     0
  3    199    21     8   108     1     1     0     0     0     0     0     0
  4    112    16     3     0    60     1     0     0     0     1     0     0
  5     41     1     1     0     0    27     0     0     0     0     0     0
  6     16     9     1     0     0     0    10     0     0     0     0     0
  7      4     2     0     0     0     0     0     2     0     0     0     0
  8      4     0     0     0     0     0     0     0     5     0     0     0
  10     3     0     0     0     0     0     0     0     0     2     0     0
  12     2     0     0     0     0     0     0     0     0     0     9     0
  18     1     1     0     0    

In [ ]:
## Data pre-processing
data2=head(data1,200000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 200000

[1] 199917

[1] 199917

'data.frame':	157077 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 15 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  30334  2160   563   273    75    38    30     5     8     3     6     0
  1   3520  2485   104    46    28     8     2     2     2     0     2     0
  2   1016   119   566    24     8     4     2     0     0     0     0     0
  3    460    47    15   243     2     1     1     0     0     0     0     0
  4    182    29     9     1    98     3     0     0     0     0     0     0
  5     95     9     7     1     0    57     1     0     0     0     0     0
  6     34    11     0     0     1     0    19     0     0     0     0     0
  7     16     2     0     0     0     0     0     6     0     0     0     0
  8      7     1     0     0     0     0     0     0    10     0     0     0
  9      0     0     0     0     0     0     0     0     0     0     0     0
  10     5     0     0     1     0     0     0     0     0     0     6     0
  11     0     1     0     0    

In [ ]:
## Data pre-processing
data2=head(data1,500000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  1                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                442                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 500000

[1] 499558

[1] 499557

'data.frame':	392508 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 16 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  76034  5362  1422   589   236   119    66    12    26     7    10     1
  1   8952  5907   304   107    59    31     8     7     3     0     3     0
  2   2618   332  1445    52    19     6     3     0     1     0     0     0
  3   1096   136    33   587     8     4     0     0     1     0     0     0
  4    439    60    19     7   201     3     2     0     0     0     1     0
  5    193    18    18     5     1   131     1     0     0     0     0     0
  6     86    15     3     0     1     0    51     0     0     0     0     0
  7     33     5     1     0     0     0     0    12     0     0     0     0
  8     23     4     1     0     0     0     0     0    23     0     0     0
  9     10     0     0     0     0     0     0     0     0     4     0     0
  10    13     0     0     0     0     0     0     0     0     0     8     0
  11     3     1     0     0    

In [ ]:
## Data pre-processing
data2=head(data1,1000000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 22                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               1106                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 1000000

[1] 998894

[1] 998872

'data.frame':	784828 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 19 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  152055  10655   2856   1150    459    221    122     36     40      9
  1   18119  11916    581    200    106     45     15     12      4      0
  2    5207    688   2768     88     33     16      3      0      1      0
  3    2124    290     69   1109     13      7      2      1      0      0
  4     878    133     40     13    442      3      3      1      2      0
  5     424     56     20     10      2    253      1      0      0      0
  6     206     31      5      0      0      2    104      0      0      0
  7      71      9      3      2      1      0      0     24      0      0
  8      43      9      0      0      2      0      0      0     37      0
  9      18      3      0      0      0      0      0      0      0      8
  10     19      3      0      0      0      0      0      0      0      0
  11      6      1      2      0      0      0      0   

In [ ]:
## Data pre-processing
data2=head(data1,2000000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 36                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               2292                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 2000000

[1] 1997708

[1] 1997672

'data.frame':	1569599 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 23 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 

Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  301398  20797   5718   2309    887    445    189     83     41     19
  1   36538  25443   1259    447    204    101     37     23      3      3
  2   10727   1386   6235    168     69     28     14      6      1      1
  3    4214    550    153   2382     23     14      2      2      0      0
  4    1797    275     64     26   1010      5      4      5      2      0
  5     846    118     33     15      5    518      1      0      1      0
  6     395     54     17      4      2      5    195      0      0      0
  7     132     16      4      2      1      0      0     96      0      0
  8      71     12      1      0      2      0      0      0     43      0
  9      40      7      0      1      0      0      0      0      0     23
  10     36     11      1      1      0      0      0      0      0      0
  11     12      1      0      0      0      0      0   

## Downsampling

In [ ]:
## Data pre-processing
data2=head(data1,100000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 100000

[1] 99917

[1] 99917

[1] 99917

[1] 37791

'data.frame':	29693 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 33 42 39 38 35 39 35 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 1 1 4 4 4 4 4 2 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 10 levels "Clear","Clouds",..: 1 1 1 1 1 

Confusion Matrix and Statistics

    predic
true    0    1    2    3    4    5    6    7    8   10   12   18
  0  1861 1000  252   94   41   12    6    0    2    1    3    0
  1   874 2155  140   53   34   10    7    3    0    4    0    0
  2   258  191  418   20    4    0    3    0    0    0    0    0
  3    96   64   16  149    3    2    1    0    0    0    0    0
  4    45   34    6    0   87    0    1    0    0    0    0    0
  5    26    8    1    2    1   38    0    0    0    0    0    0
  6    11   11    2    0    0    0   10    0    0    0    0    0
  7     4    2    0    0    0    0    0    4    0    0    0    0
  8     1    1    0    0    1    0    0    0    4    0    0    0
  10    2    1    0    0    1    0    0    0    0    1    0    0
  12    0    2    2    2    0    0    0    0    0    0    6    0
  18    0    0    1    0    0    0    0    0    0    0    0    3

Overall Statistics
                                         
               Accuracy : 0.5848         
       

In [ ]:
## Data pre-processing
data2=head(data1,200000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 200000

[1] 199917

[1] 199917

[1] 199917

[1] 72812

'data.frame':	57209 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  82 33 42 65 27 38 35 39 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 15 levels "0","1","2","3",..: 1 1 1 1 4 4 4 4 4 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 

Confusion Matrix and Statistics

    predic
true    0    1    2    3    4    5    6    7    8    9   10   11   12   16   18
  0  3503 2040  426  178   54   32   18    9    5    0    3    1    3    1    0
  1  1623 4063  227  123   39   22   19    0    2    0    3    0    0    2    0
  2   520  371  891   27    9   13    3    0    1    0    0    0    0    0    0
  3   181  161   42  354    7    5    0    0    0    0    0    0    1    0    0
  4    64   78   21    5  146    0    0    0    0    0    1    0    0    0    0
  5    37   37    8    5    2   69    0    0    0    0    0    0    0    0    0
  6    16   10    4    0    0    0   44    0    0    0    0    0    0    0    0
  7     4    5    0    0    0    0    0   10    0    0    0    0    0    0    0
  8     3    3    0    0    1    0    0    0   11    0    0    0    0    0    0
  9     0    1    0    0    0    0    0    0    0    0    0    0    0    0    0
  10    2    1    0    1    1    0    0    0    0    0    8    0    0    0  

In [ ]:
## Data pre-processing
data2=head(data1,500000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  1                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                442                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 500000

[1] 499558

[1] 499557

[1] 499557

[1] 180966

'data.frame':	142188 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 33 42 39 38 35 39 35 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 16 levels "0","1","2","3",..: 1 1 1 4 4 4 4 4 2 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0   8773  5051  1156   429   166    80    37     9     9     3     2     1
  1   4109 10117   721   307   117    48    26    14     7     2     3     0
  2   1173   910  2179    82    39    13     7     2     2     2     0     0
  3    494   362    85   805    16     8     2     2     2     0     0     0
  4    197   162    37    21   315     0     1     0     1     0     0     0
  5     64    58    20     6     2   169     0     1     0     0     0     1
  6     44    35     7     1     3     0    88     1     0     0     0     0
  7     11    11     6     3     0     0     0    19     0     0     0     0
  8     11     9     2     1     0     0     0     0    27     0     0     0
  9      1     3     0     0     0     0     0     0     0     6     0     0
  10     1     4     1     1     0     0     0     0     0     0    10     0
  11     0     1     2     0    

In [ ]:
## Data pre-processing
data2=head(data1,1000000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=100,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 22                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               1106                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 1000000

[1] 998894

[1] 998872

[1] 998872

[1] 361972

'data.frame':	284407 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  42 42 39 27 39 35 35 35 25 43 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 1 1 ...
 $ KILLED_INJURED     : Factor w/ 19 levels "0","1","2","3",..: 1 1 4 4 4 4 2 2 2 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  17348 10039  2137   805   312   153    68    27    19     4     6     3
  1   8528 20375  1423   503   226    78    43    15    11     1     6     2
  2   2463  1943  4166   171    65    33     9     4     5     0     2     1
  3   1010   811   193  1652    32    14     5     2     0     0     1     0
  4    402   305    73    17   631     5     4     6     2     0     2     0
  5    221   129    34    22     5   318     2     0     0     0     0     0
  6     91    46    12     6     6     1   146     0     0     0     0     1
  7     39    23     9     5     1     0     0    47     0     0     0     0
  8     27    28     5     2     1     0     0     0    40     0     0     0
  9      3     8     2     0     0     0     0     0     0    10     0     0
  10     3     7     1     0     0     0     0     0     0     0    13     0
  11     5     2     0     0    

# Grid Optimization

In [ ]:
## Data pre-processing
data2=head(data1,100000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
hyper_grid <- expand.grid(
  mtry       = 13,
  node_size  = 2,
  num.trees = seq(100,1000, by = 100),
  OOB_RMSE   = 0
)
for(i in 1:nrow(hyper_grid)) {
  model <- ranger(
    formula         = KILLED_INJURED~., 
    data            = training, 
    num.trees       = hyper_grid$num.trees[i],
    mtry            = hyper_grid$mtry[i],
    min.node.size   = hyper_grid$node_size[i],
    seed            = 123
  )
  
  # add OOB error to grid
  hyper_grid$OOB_RMSE[i] <- sqrt(model$prediction.error)
}
hyper_grid %>% 
  dplyr::arrange(OOB_RMSE) %>%
  head(10)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 100000

[1] 99917

[1] 99917

[1] 99917

[1] 37791

'data.frame':	29693 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 33 42 39 38 35 39 35 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 1 1 4 4 4 4 4 2 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 10 levels "Clear","Clouds",..: 1 1 1 1 1 

,mtry,node_size,num.trees,OOB_RMSE
,<dbl>,<dbl>,<dbl>,<dbl>
1,13,2,700,0.6195656
2,13,2,900,0.6196743
3,13,2,800,0.6197286
4,13,2,500,0.6198101
5,13,2,600,0.6200275
6,13,2,1000,0.6201361
7,13,2,300,0.6203804
8,13,2,400,0.6207332
9,13,2,200,0.6212213


In [ ]:
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=700,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

In [ ]:
## Data pre-processing
data2=head(data1,500000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
hyper_grid <- expand.grid(
  mtry       = 13,
  node_size  = 2,
  num.trees = seq(100,1000, by = 100),
  OOB_RMSE   = 0
)
for(i in 1:nrow(hyper_grid)) {
  model <- ranger(
    formula         = KILLED_INJURED~., 
    data            = training, 
    num.trees       = hyper_grid$num.trees[i],
    mtry            = hyper_grid$mtry[i],
    min.node.size   = hyper_grid$node_size[i],
    seed            = 123
  )
  
  # add OOB error to grid
  hyper_grid$OOB_RMSE[i] <- sqrt(model$prediction.error)
}
hyper_grid %>% 
  dplyr::arrange(OOB_RMSE) %>%
  head(10)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  1                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                442                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 500000

[1] 499558

[1] 499557

[1] 499557

[1] 180966

'data.frame':	142188 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 33 42 39 38 35 39 35 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 16 levels "0","1","2","3",..: 1 1 1 4 4 4 4 4 2 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

,mtry,node_size,num.trees,OOB_RMSE
,<dbl>,<dbl>,<dbl>,<dbl>
1,13,2,500,0.6303548
2,13,2,700,0.6304664
3,13,2,400,0.6304719
4,13,2,600,0.6305221
5,13,2,800,0.6306281
6,13,2,1000,0.6307731
7,13,2,900,0.6308399
8,13,2,300,0.6309347
9,13,2,200,0.6314305


In [ ]:
## Data pre-processing
data2=head(data1,1000000)
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
randFore100m5 <- ranger(KILLED_INJURED~., data=training, mtry=13,num.trees=500,min.node.size =2)
print(randFore100m5)
pred=predict(randFore100m5,test,type="response")
cm=table(true=test$KILLED_INJURED,predic=pred$predictions)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 22                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               1106                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 1000000

[1] 998894

[1] 998872

[1] 998872

[1] 361972

'data.frame':	284407 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 33 42 39 38 35 39 35 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 19 levels "0","1","2","3",..: 1 1 1 4 4 4 4 4 2 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  17502  9987  2193   815   345   151    61    17    23     3     9     2
  1   8480 20161  1473   569   218    97    37    21    23     3     3     1
  2   2600  1864  4142   160    65    32    10     4     2     0     1     1
  3   1011   784   199  1555    29    18     3     0     1     0     2     0
  4    378   320    75    25   683     0     7     1     2     0     0     0
  5    156   133    32    10     7   335     4     0     0     0     0     0
  6     85    64    12     2     4     0   165     1     1     0     0     0
  7     48    23     6     7     3     0     0    55     0     0     0     0
  8     19    19     1     2     2     0     0     0    41     0     0     0
  9      7     6     0     0     0     0     0     0     0    14     0     0
  10     9     5     1     1     1     0     0     0     0     0     9     0
  11     5     3     2     0    